In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DFRAME_PATH="/content/drive/MyDrive/Colab Notebooks/aggregated_df.csv"

In [4]:
df = pd.read_csv(DFRAME_PATH)

In [31]:
df.head(10)

,author,book,text,text_no_punkt,lemmas,tags,tokens
0,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мы за пять лет. Материалы [к биографии]\nКак б...,Мы за пять лет Материалы к биографии Как будто...,пять год материал биография кроваво-красный ра...,4_NUMR 3_NOUN 9_NOUN 9_NOUN 15_ADJF 6_NOUN 8_V...,Мы за пять лет . Материалы [ к биографии ] Как...
1,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Сидел он за большим письменным столом перед де...,Сидел он за большим письменным столом перед де...,сидеть больший письменный стол деревянный доск...,5_VERB 7_ADJF 10_ADJF 6_NOUN 10_ADJF 6_NOUN 6_...,Сидел он за большим письменным столом перед де...
2,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Да уж, — качал головой сдержанный Ре-ми. — Н...",Да уж качал головой сдержанный Ре-ми Нехорошо ...,качать голова сдержать ре-ми нехорошо нехорошо...,5_VERB 7_NOUN 10_PRTF 5_None 8_ADVB 8_ADVB 4_A...,"— Да уж , — качал головой сдержанный Ре-ми . —..."
3,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Поверит ли кто-нибудь, что нами за эти пять ле...",Поверит ли кто-нибудь что нами за эти пять лет...,поверить кто-нибудь пять год совместно м.г кор...,7_VERB 10_NPRO 4_NUMR 3_NOUN 9_ADVB 3_None 11_...,"Поверит ли кто-нибудь , что нами за эти пять л..."
4,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"8\nАверченко А. Избранные рассказы. М., 1985. ...",8 Аверченко А Избранные рассказы М 1985 С 7 9 ...,8 избранный рассказ м 1985 7 9 ежегодник 156 1...,1_None 9_ADJF 8_NOUN 1_NOUN 4_None 1_None 1_No...,"8 Аверченко А. Избранные рассказы . М. , 1985 ..."
5,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"— Помилуйте, такой праздник… Нужно, обязательн...",Помилуйте такой праздник Нужно обязательно поц...,помиловать праздник нужно обязательно поцелова...,9_VERB 8_NOUN 5_PRED 11_ADVB 12_INFN 7_NOUN 10...,"— Помилуйте , такой праздник… Нужно , обязател..."
6,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— Разбудить его в половине шестнадцатого.\n* *...,Разбудить его в половине шестнадцатого Закончи...,разбудить половина шестнадцатый закончить всё ...,9_INFN 8_NOUN 13_ADJF 8_GRND 3_PRCL 6_NOUN 7_N...,— Разбудить его в половине шестнадцатого . * *...
7,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,"Он встал, оделся, побрился и вышел на улицу.\n...",Он встал оделся побрился и вышел на улицу На у...,встать одеться побриться выйти улица улица сто...,5_VERB 6_VERB 8_VERB 5_VERB 5_NOUN 5_NOUN 11_V...,"Он встал , оделся , побрился и вышел на улицу ..."
8,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,Мозаика\nI\n— Я несчастный человек — вот что!\...,Мозаика I Я несчастный человек вот что Что за ...,мозаика i несчастный человек вздор поверить ув...,7_NOUN 1_None 10_ADJF 7_NOUN 5_NOUN 6_VERB 6_V...,Мозаика I — Я несчастный человек — вот что ! —...
9,Аверченко,Averchenko_A-T-Averchenko-Sobranie-sochineniy-...,— Как записываешь?\n— В записной книжке. Хочеш...,Как записываешь В записной книжке Хочешь У мен...,записывать записной книжка хотеть минута откро...,11_VERB 8_ADJF 6_NOUN 6_VERB 6_NOUN 13_NOUN 6_...,— Как записываешь ? — В записной книжке . Хоче...


In [ ]:
df.shape

(17736, 7)

In [5]:
def get_document_vectorizer(frame, n_min=1, n=2, max_count=10000, column="text"):
    texts_vector = frame[column]
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    vectorizer.fit(texts_vector)
    return vectorizer

def get_author_vectorizer(frame, n_min=1, n=2, max_count=10000, column="text"):
    grouped_text = frame.groupby("author", as_index = False)[column].agg({column: ' '.join})
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n), max_features=max_count, norm='l2')
    texts_vector = frame[column]
    vectorizer.fit(texts_vector)
    return vectorizer

In [6]:
def get_encoder(frame, column="author"):
    encoder = LabelEncoder()
    encoder.fit(frame[column])
    return encoder

### Попробуем в лоб:
возьмем лемматизированные отрезки текста, векторизируем их и попробуем предсказать

In [7]:
label_encoder = get_encoder(frame=df)

In [9]:
y = label_encoder.transform(df["author"])
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.33)

In [11]:
vectorizer = get_author_vectorizer(df_train, max_count=20000)
x_train = vectorizer.transform(df_train["text"])
x_test = vectorizer.transform(df_test["text"])

In [ ]:
clf = LogisticRegression(class_weight="balanced",
                        max_iter=500)
clf.fit(x_train, y_train)

In [ ]:
f1_score(clf.predict(x_test), y_test, average="micro")

0.971622641509434

Если играться с параметрами, точность колеблется около 96-97%

### Попробуем на частях речи

In [ ]:
label_encoder = get_encoder(frame=df)
y = label_encoder.transform(df["author"])
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.5)

In [ ]:
vectorizer = get_author_vectorizer(df_train, max_count=10000, column="tags", n_min=1, n=3)
x_train = vectorizer.transform(df_train["tags"])
x_test = vectorizer.transform(df_test["tags"])

In [ ]:
clf = LogisticRegression(class_weight="balanced",
                         warm_start=False,
                         max_iter=500)
clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [ ]:
f1_score(clf.predict(x_test), y_test, average="micro")

0.6599245283018867

In [ ]:
f1_score(clf.predict(x_train), y_train, average="micro")

0.8221618357487923

Через перебор n-gram и длины вектора точность на тесте не выше 0.66

### комбинируем

In [ ]:
vectorizer1 = get_author_vectorizer(df_train, max_count=30000)
vectorizer2 = get_author_vectorizer(df_train, max_count=5000, column="tags", n_min=1, n=3)

x_train1 = vectorizer1.transform(df_train["lemmas"])
x_train2 = vectorizer2.transform(df_train["tags"])

x_test1 = vectorizer1.transform(df_test["lemmas"])
x_test2 = vectorizer2.transform(df_test["tags"])

x_train = sp.sparse.hstack([x_train1, x_train2])
x_test = sp.sparse.hstack([x_test1, x_test2])

In [ ]:
clf = LogisticRegression(class_weight="balanced",
                         warm_start=False,
                         max_iter=500)
clf.fit(x_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500)

In [ ]:
f1_score(clf.predict(x_test), y_test, average="micro")

0.9559245283018868

комбинация ничего особо не дает. Не похоже на переобучение, при снижении количества параметров мы ничего особо не выигрываем

Если поможет то гуд 
TfIdf для нескольких колонок

In [ ]:
TfIdf для нескольких колонок
In [9]:
class MultiTfidf(TransformerMixin):
    def __init__(self, cols=None, tfidf_type='classic'):
        self.cols = cols
        self.tfidf_type = tfidf_type
    
    def fit(self, X, y=None):
        if self.tfidf_type == 'classic':
            get_vectorizer = get_document_vectorizer
        elif self.tfidf_type == 'class_based':
            get_vectorizer = get_author_vectorizer
        else:
            raise ValueError("Unknown vectorizer")
        self.vectorizers_ = [get_vectorizer(X, column=col) for col in self.cols]
        return self
    
    def transform(self, X, y=None):
        results = []
        for vec, col in zip(self.vectorizers_, self.cols):
            results.append(vec.transform(X[col]))
        return sp.hstack(results)

базовый пайплайн

In [ ]:
def get_base_estimator(cols, vec_type='classic'):
    pipe = Pipeline([
        ('vectorizer', MultiTfidf(cols=cols, tfidf_type=vec_type)),
        ('model', LogisticRegression(class_weight='balanced', max_iter=500, C=1000))
    ])
    return pipe

## CATBOOST

In [12]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [14]:
from catboost import Pool, CatBoostClassifier

In [15]:
df = df[["text"]]


In [17]:
df_train, df_test, y_train, y_test = train_test_split(df, y, train_size=0.33)

In [33]:
train_pool = Pool(
    df_train, 
    y_train, 
    text_features=["text"],
    
   
)
valid_pool = Pool(
    df_test, 
    y_test,
    text_features=["text"], 
   
)

catboost_params = {
    'iterations': 20000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'task_type': 'GPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 2000,
    'l2_leaf_reg': 1
    
}
model = CatBoostClassifier(**catboost_params,random_seed = 43)



In [34]:
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.6925837	test: 0.7470549	best: 0.7470549 (0)	total: 35.9ms	remaining: 11m 57s
2000:	learn: 0.9519822	test: 0.9484180	best: 0.9485022 (1920)	total: 42.8s	remaining: 6m 25s
4000:	learn: 0.9854751	test: 0.9518681	best: 0.9521205 (3890)	total: 1m 27s	remaining: 5m 51s
6000:	learn: 0.9981203	test: 0.9542242	best: 0.9543083 (5788)	total: 2m 8s	remaining: 4m 59s
8000:	learn: 0.9998291	test: 0.9559071	best: 0.9559912 (7825)	total: 2m 47s	remaining: 4m 11s
10000:	learn: 1.0000000	test: 0.9568327	best: 0.9568327 (9960)	total: 3m 28s	remaining: 3m 28s
12000:	learn: 1.0000000	test: 0.9575900	best: 0.9577583 (11437)	total: 4m 8s	remaining: 2m 45s
14000:	learn: 1.0000000	test: 0.9580949	best: 0.9583474 (13444)	total: 4m 52s	remaining: 2m 5s
16000:	learn: 1.0000000	test: 0.9585157	best: 0.9585157 (14791)	total: 5m 32s	remaining: 1m 23s
18000:	learn: 1.0000000	test: 0.9587681	best: 0.9587681 (16494)	total: 6m 12s	remaining: 41.4s
19999:	learn: 1.0000000	test: 0.9590205	best: 0.9591047 (1982